In [1]:
import pandas as pd
import numpy as np

In [2]:
file = ('houses.csv')
df = pd.read_csv(file)
df.head()

,Unnamed: 0,Location,Property type,Property subtype,Price,Type of sale,Number of bedrooms,Living area,Kitchen,Furnished,Open fireplace,Terrace,Terrace orientation,Garden,Garden orientation,Surface area land,Number of facades,Pool,Condition
0,0,Verviers,HOUSE,HOUSE,296607.0,residential_sale,3.0,130.0,Unknown,False,False,True,Unknown,True,Unknown,239.0,3,NaN,AS_NEW
1,1,Haasdonk,HOUSE,HOUSE,560000.0,residential_sale,4.0,NaN,Unknown,NaN,False,Unknown,Unknown,Unknown,Unknown,626.0,3,NaN,AS_NEW
2,2,VIANE,HOUSE,HOUSE,299000.0,residential_sale,5.0,200.0,SEMI_EQUIPPED,False,False,Unknown,Unknown,Unknown,Unknown,1150.0,NaN,NaN,GOOD
3,3,Gerpinnes,HOUSE,HOUSE,300000.0,residential_sale,0.0,270.0,Unknown,False,False,True,SOUTH,True,SOUTH,498.0,3,False,NaN
4,4,Nivelles,HOUSE,HOUSE,195000.0,residential_sale,3.0,135.0,INSTALLED,False,False,True,Unknown,Unknown,Unknown,165.0,2,False,NaN


In [3]:
df.drop(['Unnamed: 0','Furnished','Terrace orientation','Garden orientation','Number of facades'], axis =1, inplace=True)
df.head()

,Location,Property type,Property subtype,Price,Type of sale,Number of bedrooms,Living area,Kitchen,Open fireplace,Terrace,Garden,Surface area land,Pool,Condition
0,Verviers,HOUSE,HOUSE,296607.0,residential_sale,3.0,130.0,Unknown,False,True,True,239.0,NaN,AS_NEW
1,Haasdonk,HOUSE,HOUSE,560000.0,residential_sale,4.0,NaN,Unknown,False,Unknown,Unknown,626.0,NaN,AS_NEW
2,VIANE,HOUSE,HOUSE,299000.0,residential_sale,5.0,200.0,SEMI_EQUIPPED,False,Unknown,Unknown,1150.0,NaN,GOOD
3,Gerpinnes,HOUSE,HOUSE,300000.0,residential_sale,0.0,270.0,Unknown,False,True,True,498.0,False,NaN
4,Nivelles,HOUSE,HOUSE,195000.0,residential_sale,3.0,135.0,INSTALLED,False,True,Unknown,165.0,False,NaN


In [4]:
df = df[df["Property type"]== "HOUSE"]

df = df.replace({'Property subtype' : {'BUNGALOW':'HOUSE', 'TOWN_HOUSE': 'HOUSE', 'COUNTRY_COTTAGE': 'HOUSE','CHALET':'HOUSE','COUNTRY_COTTAGE':'HOUSE'}}, regex=True)
df = df[~df["Property subtype"].isin(["APARTMENT_BLOCK","CASTLE","EXCEPTIONAL_PROPERTY","HOUSE_GROUP","OTHER_PROPERTY","MANOR_HOUSE"])]

df = df[df["Type of sale"]== "residential_sale"]
df = df[(df["Number of bedrooms"]<10) & df["Number of bedrooms"]>0 ]

df = df[df["Surface area land"]< 5000]

df = df.replace({'Kitchen' : {'USA_HYPER_EQUIPPED':True,'USA_INSTALLED':True,'USA_SEMI_EQUIPPED':True,'USA_UNINSTALLED':False,'INSTALLED':True,'Unknown':False,'SEMI_EQUIPPED':False,'HYPER_EQUIPPED':True,'NOT_INSTALLED':False, np.nan:False}}, regex=True)

df = df.replace({'Terrace' : {'Unknown':'False'}}, regex=True)
df = df.replace({'Garden' : {'Unknown':'False'}}, regex=True)

df = df.replace({'Pool' : {np.nan:'False'}}, regex=True)

# 1 means renovation needed
df = df.replace({'Condition' : {'GOOD':False, 'AS_NEW':False, 'JUST_RENOVATED':False,'TO_RENOVATE': True, 'TO_BE_DONE_UP':True,'Unknown':True,'TO_RESTORE':True, np.nan:True, }}, regex=True)

In [5]:
df.dropna(subset=['Price', 'Number of bedrooms','Living area'], inplace=True)

In [6]:
df.reset_index(inplace=True)
df = pd.get_dummies(df, columns=['Property subtype','Kitchen','Open fireplace','Terrace','Garden','Pool','Condition'])
df.drop(['Property type','Type of sale','index','Kitchen_False','Open fireplace_False','Terrace_False','Garden_False','Pool_False','Condition_False'], axis =1, inplace=True)
df.rename(columns= {'Kitchen_True':'Kitchen equipped','Open fireplace_True':'Open fireplace','Terrace_True':'Terrace','Garden_True':'Garden','Pool_True':'Pool','Condition_True':'Condition'}, inplace = True)

In [7]:
df

,Location,Price,Number of bedrooms,Living area,Surface area land,Property subtype_FARMHOUSE,Property subtype_HOUSE,Property subtype_MANSION,Property subtype_MIXED_USE_BUILDING,Property subtype_VILLA,Kitchen equipped,Open fireplace,Terrace,Garden,Pool,Condition
0,Verviers,296607.0,3.0,130.0,239.0,0,1,0,0,0,0,0,1,1,0,0
1,VIANE,299000.0,5.0,200.0,1150.0,0,1,0,0,0,0,0,0,0,0,0
2,Nivelles,195000.0,3.0,135.0,165.0,0,1,0,0,0,1,0,1,0,0,1
3,Dinant,175000.0,3.0,128.0,260.0,0,1,0,0,0,1,0,1,0,0,1
4,Lierde,415000.0,3.0,207.0,534.0,0,1,0,0,0,1,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6883,Zaventem,488000.0,3.0,150.0,390.0,0,1,0,0,0,0,0,1,1,0,0
6884,Aalst,550000.0,4.0,1380.0,1300.0,0,0,0,0,1,1,1,1,1,0,0
6885,Charleroi Ransart,115000.0,2.0,120.0,310.0,0,1,0,0,0,1,0,0,1,0,1
6886,Sars-la-Bruyère,280000.0,3.0,120.0,839.0,0,0,0,0,1,1,0,1,1,0,0


In [8]:
#postcodes = pd.read_csv('postcodes_filtered.csv')
#postcodes.head()

In [9]:
"""for index, row in df.iterrows():
    place = row['Location']
    print(place)
    for num, line in postcodes.iterrows():
        gemeente = line['Gemeente']
        if place == line:
            print('yes')
        else:
            print('no')
   """ 

"for index, row in df.iterrows():\n    place = row['Location']\n    print(place)\n    for num, line in postcodes.iterrows():\n        gemeente = line['Gemeente']\n        if place == line:\n            print('yes')\n        else:\n            print('no')\n   "

In [10]:
df.to_csv('houses_filtered.csv', index = False)